In [ ]:
import pandas as pd
import boto3
import json

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')
DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

# redshit cluster parameters
DWH_CLUSTER_TYPE       = config.get("CLUSTER","DB_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("CLUSTER","DB_NUM_NODES")
DWH_NODE_TYPE          = config.get("CLUSTER","DB_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("CLUSTER","DB_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("CLUSTER","DB_NAME")
DWH_DB_USER            = config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DB_PORT")


In [ ]:
s3 = boto3.resource(
    service_name='s3',
    region_name='us-west-2',
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET
)

In [ ]:
sampleDbBucket =  s3.Bucket("udacity-dend")
print(type(sampleDbBucket))

In [ ]:
# Print out bucket names
for bucket in sampleDbBucket.objects.all():
    print(bucket)

In [ ]:


obj_list = list(sampleDbBucket.objects.filter(Prefix='song_data/A/B/C/'))
obj = obj_list[0]
obj

In [ ]:
df_song_data = pd.read_json('s3://udacity-dend/song_data/A/B/C/TRABCAS128F14A25E2.json',  lines=True)
df_song_data

In [ ]:
obj_list = list(sampleDbBucket.objects.filter(Prefix='log_data'))
obj = obj_list[1]
obj

In [ ]:
df_log_data = pd.read_json('s3://udacity-dend/log_data/2018/11/2018-11-01-events.json',  lines=True)
df_log_data.head()

In [ ]:
s3 = boto3.client('s3')

Create an IAM role, attach policy and get ARN (amazon resource name) programatically

In [ ]:
iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

In [ ]:
from botocore.exceptions import ClientError

#1.1 Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

Create Redshift cluster with parameters from the config file and Attach IAM role 

In [ ]:
# create redshit client
redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

In [ ]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

In [ ]:
# cluster parameters

def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
# cluster end point and ARN

DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

Open an incoming TCP port to access the cluster ednpoint

In [ ]:
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        # GroupName=defaultSg.group_name,
        GroupName='default',
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

Connect to the RedShift Cluster

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
%store conn_string
print(conn_string)
%sql $conn_string

create staging tables for song and log file. Use COPY command to get data from S3 to Redshift

In [ ]:
%%sql

DROP TABLE IF EXISTS "staging_songs";
DROP TABLE IF EXISTS "staging_events_log";

CREATE TABLE staging_songs(
        num_songs           INTEGER,
        artist_id           VARCHAR,
        artist_latitude     FLOAT,
        artist_longitude    FLOAT,
        artist_location     VARCHAR,
        artist_name         VARCHAR,
        song_id             VARCHAR,
        title               VARCHAR,
        duration            FLOAT,
        year                INTEGER
    );

CREATE TABLE staging_events_log(
    artist                  VARCHAR,
    auth                    VARCHAR,
    firstName               VARCHAR,              
    gender                  VARCHAR,
    itemInSession           INTEGER,
    lastName                VARCHAR,
    length                  FLOAT,
    level                   VARCHAR,
    location                VARCHAR,
    method                  VARCHAR,
    page                    VARCHAR,
    registration            FLOAT,
    sessionId               INTEGER,
    song                    VARCHAR,
    status                  INTEGER,
    ts                      TIMESTAMP,
    userAgent               VARCHAR,
    userId                  INTEGER
);


In [ ]:
# copy from S3 to songs staging table
qry = """
    copy staging_songs 
    from {} \
    iam_role {}
    json 'auto';
""".format(config['S3']['SONG_DATA'],config['IAM_ROLE']['ARN'])

%sql $qry

In [ ]:
%sql SELECT count(*) FROM staging_songs;

In [ ]:
%sql SELECT * FROM staging_songs LIMIT 10;

In [ ]:
# copy from S3 to events log staging table
qry = """
    copy staging_events_log 
    from {} \
    iam_role {}\
    FORMAT AS json {}\
    TIMEFORMAT 'epochmillisecs';
""".format(config['S3']['LOG_DATA'],config['IAM_ROLE']['ARN'],config['S3']['LOG_JSONPATH'])

%sql $qry

In [ ]:
%sql SELECT count(*) FROM staging_events_log;

In [ ]:
%sql SELECT * FROM staging_events_log LIMIT 5;

Create Dimension and Fact Table. 
Get data from staging table to target table using INSERT/INSERT INTO

In [ ]:
%%sql

DROP TABLE IF EXISTS "song_table";
DROP TABLE IF EXISTS "artist_table";
DROP TABLE IF EXISTS "user_table";
DROP TABLE IF EXISTS "time_table";
DROP TABLE IF EXISTS "songplay_table";

CREATE TABLE song_table(
    song_id     VARCHAR, 
    title       VARCHAR, 
    artist_id   VARCHAR, 
    year        INT,
    duration    FLOAT
);

CREATE TABLE artist_table(
    artist_id   VARCHAR,
    name        VARCHAR, 
    location    VARCHAR, 
    latitude    FLOAT, 
    longitude   FLOAT
);

CREATE TABLE user_table(
    user_id     INTEGER, 
    first_name  VARCHAR, 
    last_name   VARCHAR, 
    gender      VARCHAR, 
    level       VARCHAR
);

CREATE TABLE time_table(
    start_time      TIMESTAMP,
    time_hour       INTEGER, 
    time_day        INTEGER, 
    time_week       INTEGER, 
    time_month      INTEGER, 
    time_year       INTEGER, 
    time_weekday    INTEGER
);

CREATE TABLE songplay_table(
    songplay_id     INT IDENTITY(0,1), 
    start_time      VARCHAR, 
    user_id         INTEGER, 
    level           VARCHAR, 
    song_id         VARCHAR, 
    artist_id       VARCHAR, 
    session_id      INTEGER, 
    location        VARCHAR, 
    user_agent      VARCHAR
);


Test songplay query

In [ ]:
%%sql

SELECT ts, userId, level, song_id, artist_id, sessionId, location, userAgent FROM staging_events_log se, staging_songs ss 
WHERE se.page='NextSong' 
AND se.song = ss.title
AND se.artist = ss.artist_name
AND se.length = ss.duration
limit 5;

Ingest from staging tables to target tables. Use INSERT INTO SELECT

In [ ]:
%%sql

INSERT INTO song_table SELECT song_id, title, artist_id, year, duration FROM staging_songs;

INSERT INTO artist_table SELECT artist_id, artist_name, artist_location, artist_latitude, artist_longitude FROM staging_songs;

INSERT INTO user_table SELECT userId, firstName, lastName, gender, level FROM staging_events_log;

INSERT INTO time_table SELECT ts, EXTRACT(HOUR FROM ts), EXTRACT(DAY FROM ts), EXTRACT(WEEK FROM ts), EXTRACT(MONTH FROM ts), EXTRACT(YEAR FROM ts), EXTRACT(dayofweek FROM ts) FROM staging_events_log;

INSERT INTO songplay_table (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) SELECT ts, userId, level, song_id, artist_id, sessionId, location, userAgent FROM staging_events_log se, staging_songs ss 
WHERE se.page='NextSong' 
AND se.song = ss.title
AND se.artist = ss.artist_name
AND se.length = ss.duration;


In [ ]:
%sql SELECT * FROM time_table LIMIT 5;

Delete redshit cluster (DO NOT run this during execution)

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

Delete IAM role

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
# iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
# iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!